In [1]:
import apache_beam as beam


In [2]:
class SplitRow(beam.DoFn):
    def process(self, element):
        return [element.split(',')]
    
class FilterAccountsEmployee(beam.DoFn):
    def process(self, element):
        if element[3] == 'Accounts':
            return [element]
        
class PairEmployees(beam.DoFn):
    def process(self, element):
        return [(element[3]+","+element[1],1)]
    
class Counting(beam.DoFn):
    def process(self, element):
        (key, values) = element
        return [(key,sum(values))]

In [4]:
p1 = beam.Pipeline()

attendance_count = (
    p1
    |beam.io.ReadFromText('dept_data.txt')
    |beam.ParDo(SplitRow())
    |beam.ParDo(FilterAccountsEmployee())
    |beam.ParDo(PairEmployees())
#     | beam.Filter(lambda record: (record[3]+" "+record[1],1))
    |'Group '>> beam.GroupByKey()
    |'Sum using ParDo'>> beam.ParDo(Counting())
    |beam.io.WriteToText('data/output_ParDo')
)

p1.run()

!{('head -n 50 data/output_ParDo-00000-of-00001')}

('Accounts,Marco', 31)
('Accounts,Rebekah', 31)
('Accounts,Itoe', 31)
('Accounts,Edouard', 31)
('Accounts,Kyle', 62)
('Accounts,Kumiko', 31)
('Accounts,Gaston', 31)
('Accounts,Ayumi', 30)
